In [1]:
%matplotlib notebook

import torch
import numpy as np
import matplotlib.pyplot as plt
from model import MLP 

from utils import viz_samples
from config import Config
from noise_scheduler import NoiseScheduler
from utils import get_dataset

In [2]:
cfg = Config()
log_dir = cfg.log_dir
ns = NoiseScheduler(cfg)
model = MLP(cfg).cuda()
model.load_state_dict(torch.load(cfg.log_dir + "params.pt"))
model.eval()

MLP(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [3]:
# '''
# Run inference
# '''
# n = 1000
# x_last = torch.randn(n, 2).cuda()
# samples = [x_last]
# 
# for t in reversed(range(cfg.timesteps)):
#     with torch.no_grad():
#         x_t = samples[-1]
#         eps_pred = model(x_t, t)
#         # reconstruct image here
#     x_old = ns.remove_noise(x_t, eps_pred, t)
#     # add to samples
#     samples.append(x_old)

In [4]:
# samples[0].shape

In [5]:
# x_0 = samples[0][:, 0].cpu()
# y_0 = samples[0][:, 1].cpu()
# 
# offset = 0
# x_t = samples[ns.num_timesteps-offset][:, 0].cpu()
# y_t = samples[ns.num_timesteps-offset][:, 1].cpu()
# 
# plt.figure()
# 
# plt.subplot(1,2,1)
# #plt.axis([-5, 5, -5, 5])
# plt.title("t = {}".format(0))
# plt.scatter(x_0, y_0)
# 
# plt.subplot(1,2,2)
# plt.title("t = {}".format(ns.num_timesteps))
# plt.scatter(x_t, y_t)

In [6]:
from utils import viz_sample, viz_samples, get_dataset

In [7]:
x_0 = get_dataset(cfg).tensors[0].cuda()  # original data
ns = NoiseScheduler(cfg)
samples = [x_0]
for t in range(cfg.timesteps):
    noise = torch.randn(x_0.shape).cuda()
    res = ns.add_noise(x_0, t, noise)
    samples.append(res)
viz_samples([v.cpu() for v in samples])  # you can control timestep via slider

<IPython.core.display.Javascript object>

In [8]:
#log_dir = "assets/learning_examples/cat"  # set your directory: learning_results/yyyy-mm-dd_hh-mm-ss-xx
eval_data_size = 1000
#ns, model = reconstruct(log_dir)
model.eval()
x_last = torch.randn(eval_data_size, 2).cuda()
samples = [x_last]
for t in reversed(range(cfg.timesteps)):
    with torch.no_grad():
        residual = model(samples[-1], t)
    samples.append(ns.remove_noise(samples[-1], residual, t))
viz_samples([v.cpu() for v in samples][::-1])  # you can control timestep via slider

tensor([[0.5661, 0.9118],
        [0.7662, 0.7571],
        [0.9530, 0.5683],
        ...,
        [0.1433, 0.8791],
        [0.4533, 0.9400],
        [0.4517, 0.0024]], device='cuda:0')

tensor([[ 1.5105,  0.1630],
        [ 0.4811,  0.7419],
        [-0.0961,  0.3270],
        ...,
        [ 1.7441,  0.4081],
        [-1.1476,  0.3916],
        [-0.5370, -1.0294]], device='cuda:0')
tensor([[0.6212, 0.1516],
        [0.5235, 0.2400],
        [0.4606, 0.8464],
        ...,
        [0.0710, 0.9013],
        [0.8856, 0.4194],
        [0.9578, 0.1719]], device='cuda:0')

tensor([[-2.1502, -0.6047],
        [ 0.9085, -0.4569],
        [-1.2886, -1.9397],
        ...,
        [-0.4198,  1.1922],
        [-0.0252, -1.4348],
        [ 1.6508,  1.2845]], device='cuda:0')
tensor([[0.0454, 0.4595],
        [0.1857, 0.3871],
        [0.7488, 0.8170],
        ...,
        [0.5791, 0.9740],
        [0.9882, 0.0493],
        [0.4750, 0.1165]], device='cuda:0')

tensor([[ 1.0695,  0.6495],
        [ 

<IPython.core.display.Javascript object>